In [1]:
import os
os.chdir('C:\\Master_Code_92589')

In [2]:
from ctypes import *
import random
from os import listdir
from os.path import isfile, isdir, join
import time
from queue import Queue
from collections import defaultdict
import csv
import pandas as pd
import numpy as np
import ast
import json
from PIL import Image
#import imagehash
import hashlib
import math
from statsmodels.tsa.api import ExponentialSmoothing, Holt, SimpleExpSmoothing

In [3]:
table = np.array([[ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],\
        [ 0, 1, 0, 0, 0, 0,11, 0, 0, 0, 0,21],\
        [ 0, 2, 0, 0, 0, 0,12, 0, 0, 0, 0,22],\
        [ 0, 3, 0, 0, 0, 0,13, 0, 0, 0, 0,23],\
        [ 0, 4, 0, 0, 0, 0,14, 0, 0, 0, 0,24],\
        [ 0, 5, 0, 0, 0, 0,15, 0, 0, 0, 0,25],\
        [ 0, 6, 0, 0, 0, 0,16, 0, 0, 0, 0,26],\
        [ 0, 7, 0, 0, 0, 0,17, 0, 0, 0, 0,27],\
        [ 0, 8, 0, 0, 0, 0,18, 0, 0, 0, 0,28],\
        [ 0, 9, 0, 0, 0, 0,19, 0, 0, 0, 0,29],\
        [ 0,10, 0, 0, 0, 0,20, 0, 0, 0, 0,30],\
        [ 0,31,32,33,34,35,36,37,38,39,40,41] ], dtype = int)

In [4]:
scripted_walk = [
    'one_way_walk_1','one_way_walk_2','one_way_walk_3','one_way_walk_4','one_way_walk_5','one_way_walk_6','one_way_walk_7','one_way_walk_8',\
    'round_trip_walk_2','round_trip_walk_3','round_trip_walk_4'
]
xscripted_walk = [
    'one_way_walk_1','one_way_walk_2','one_way_walk_4','one_way_walk_7','one_way_walk_8',\
    'round_trip_walk_2','round_trip_walk_3','round_trip_walk_4'
]

stationary = [
    'stationary_1'
]

Xfreewalk = [
    'freewalk_1','freewalk_2','freewalk_3','freewalk_4','freewalk_5','freewalk_6','freewalk_7','freewalk_8','freewalk_9'
]
freewalk = [
    'freewalk_1','freewalk_3','freewalk_4','freewalk_5','freewalk_6','freewalk_8','freewalk_9'
]


walk_list = scripted_walk


methods_list =  [f'1_Hashing_hTCU19e',f'1_Hashing_sharp025',f'1_Hashing_GalaxyA51',f'1_Hashing_hTCU11',\
            f'2_Hashing_hTCU19e+sharp025',f'2_Hashing_hTCU19e+GalaxyA51',f'2_Hashing_sharp025+GalaxyA51',\
            f'3_Hashing_hTCU19e+sharp025+GalaxyA51',f'4_Hashing_ALL',f'4_image_Hashing'
]

methods = methods_list[8]

In [5]:
def signed_encode(hash_):
    # signed compute
    hash_ = np.where(hash_ > 0, hash_, 0)
    hash_ = np.where(hash_ <= 0, hash_, 1)
    
    return hash_.astype(int)


def text_hash(beacon_id, RSSI):
    #print(type(beacon_id))
    #print(type(RSSI))
    if walk_list == freewalk :
        RSSI_2 = 3*RSSI*RSSI + 2*RSSI*RSSI*RSSI + 1*RSSI*RSSI*RSSI*RSSI  #freewalk
    if walk_list == scripted_walk :
        RSSI_2 = 1*RSSI + 3*RSSI*RSSI*RSSI + 2*RSSI*RSSI*RSSI*RSSI       #scripted_walk
    #RSSI_2 = 3*RSSI*RSSI*RSSI
    weight = abs(RSSI_2*10)
    
    text_ = hashlib.blake2b(beacon_id.encode()).hexdigest()
    #text_ = hashlib.sha256(beacon_id.encode()).hexdigest()
    text_encode = np.array([])
    
    for x in text_:
        value_ = int(x, base=16)
        tmp = '{0:04b}'.format(value_)
        value_list = [int(char) for char in tmp]
        text_encode = np.append(text_encode, value_list)
        text_encode = text_encode.astype(int)

    # weighted
    text_encode = np.where(text_encode < 0, text_encode, text_encode*weight)
    text_encode = np.where(text_encode > 0, text_encode, -1*weight)
    
    return text_encode

def similarity(hash1_, hash2_):
    #print(hash1_)
    #print(hash2_)
    #print(abs(hash1_-hash2_).sum())
    #print('len of hash1:',len(hash1_))
    #return 1 - abs(hash1_-hash2_).sum()/len(hash1_)
    return 1 - (((abs(hash1_- hash2_)).sum())/len(hash1_))

In [6]:
#Wireless Train Hashing
name_type = 'beacon'
wireless_path = f'./data/0318_92589_train/'
Wireless_Train = pd.read_csv(join(wireless_path, 'wireless_training_set.csv'))

list_of_Wireless_Train_hash = []
train_label = []
    
for i in range(len(Wireless_Train)):
    Wireless_Train_row = Wireless_Train.iloc[i].to_dict()
    Wireless_Train_row_label = Wireless_Train_row['label']
    train_label.append(Wireless_Train_row_label)
    Wireless_Train_row.pop('label',None)

    # device1 hash encode
    Wireless_Train_hash = np.array([])
    for beacon_id, RSSI in Wireless_Train_row.items():
        hash_ = text_hash(beacon_id, RSSI)
        if len(Wireless_Train_hash) < 1:
            Wireless_Train_hash = hash_
        else:
            Wireless_Train_hash = Wireless_Train_hash + hash_
           
    Wireless_Train_hash = signed_encode(Wireless_Train_hash)
    list_of_Wireless_Train_hash.append(Wireless_Train_hash)

In [7]:
#Test Hashing 一起討論後推舉出候選人投票 寫給 image &pf
list_MDE = []
total_DE = 0
len_of_all_label = 0
K = 45

error_distri = [0]*16

for walk in walk_list:
    print("Walk Method : ",walk)
    hash_similarity = []
    #list_of_Wireless_Train_hash_dic = list(Wireless_Train_hash_dic.values())

    test_label = []
    predict_label = []

    file_name = f'./92589_wireless_{methods}_{walk}.csv'
    with open(file_name, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['label',methods])
        
        Wireless_Test = pd.read_csv(join(f'./Data/0318_92589_test/{walk}/GalaxyA51/wireless_fingerprint_avg_10_7_beacon_rate_10.csv'))
        #Wireless_Test = pd.read_csv(join(f'./Data/0318_92589_test/{walk}/hTCU11/wireless_fingerprint_avg_10_6_beacon_rate_10.csv'))
        #Wireless_Test = pd.read_csv(join(f'./Data/0318_92589_test/{walk}/hTCU19e/wireless_fingerprint_avg_10_6_beacon_rate_10.csv'))
        #Wireless_Test = pd.read_csv(join(f'./Data/0318_92589_test/{walk}/sharp025/wireless_fingerprint_avg_10_6_beacon_rate_10.csv'))
        Wireless_Test_device2 = pd.read_csv(join(f'./Data/0318_92589_test/{walk}/hTCU11/wireless_fingerprint_avg_10_7_beacon_rate_10.csv'))
        Wireless_Test_device3 = pd.read_csv(join(f'./Data/0318_92589_test/{walk}/hTCU19e/wireless_fingerprint_avg_10_7_beacon_rate_10.csv'))
        Wireless_Test_device4 = pd.read_csv(join(f'./Data/0318_92589_test/{walk}/sharp025/wireless_fingerprint_avg_10_7_beacon_rate_10.csv'))
        
        wireless_reference_point_path = f'./Result/{methods}_{walk}_wireless_reference_point.csv'
        with open(wireless_reference_point_path, 'w', newline='') as csvfile_2 :
            wireless_reference_point = csv.writer(csvfile_2)
                
            header = []
            header.append('label')
            header.append('wireless_reference_point')
            header.append('voter_wireless_reference_point')

            wireless_reference_point.writerow(header)
            for j in range(len(Wireless_Test)):
        #Device1
                Wireless_Test_row = Wireless_Test.iloc[j].to_dict()
                Wireless_Test_row_label = Wireless_Test_row['label']
                test_label.append(Wireless_Test_row_label)
                Wireless_Test_row.pop('label',None)       

                # device1 hash encode
                Wireless_Test_hash = np.array([])            
                for beacon_id, RSSI in Wireless_Test_row.items():
                    hash_ = text_hash(beacon_id, RSSI)
                    if len(Wireless_Test_hash) < 1:
                        Wireless_Test_hash = hash_
                    else:
                        Wireless_Test_hash = Wireless_Test_hash + hash_
        #Device1

        #Device2            
                Wireless_Test_row_device2 = Wireless_Test_device2.iloc[j].to_dict()
                Wireless_Test_row_device2.pop('label',None)

                # device2 hash encode           
                for beacon_id, RSSI in Wireless_Test_row_device2.items():
                    hash_ = text_hash(beacon_id, RSSI)
                    Wireless_Test_hash = Wireless_Test_hash + hash_ #hTCU11
        #Device2

        #Device3            
                Wireless_Test_row_device3 = Wireless_Test_device3.iloc[j].to_dict()
                Wireless_Test_row_device3.pop('label',None)

                # device3 hash encode           
                for beacon_id, RSSI in Wireless_Test_row_device3.items():
                    hash_ = text_hash(beacon_id, RSSI)
                    Wireless_Test_hash = Wireless_Test_hash + hash_  #hTCU19e
        #Device3

        #Device4            
                Wireless_Test_row_device4 = Wireless_Test_device4.iloc[j].to_dict()
                Wireless_Test_row_device4.pop('label',None)

                # device4 hash encode           
                for beacon_id, RSSI in Wireless_Test_row_device4.items():
                    hash_ = text_hash(beacon_id, RSSI)
                    Wireless_Test_hash = Wireless_Test_hash + hash_  #sharp025
        #Device4

                Wireless_Test_hash = signed_encode(Wireless_Test_hash)

                #找出 Top K 個像的
                k_top_similarity = [0.0]*K
                voter = [0.0]*K
                for k in range(len(list_of_Wireless_Train_hash)) :
                    sim_ = similarity(list_of_Wireless_Train_hash[k], Wireless_Test_hash)
                    if sim_ > (min(k_top_similarity)) :
                        k_top_similarity[k_top_similarity.index(min(k_top_similarity))] = sim_
                        voter[k_top_similarity.index(min(k_top_similarity))] = train_label[k]

                #for m in range(len(voter_device2)):
                    #voter.append(voter_device2[m])

                #將 similarity 做為 voter 的加權
                vote = [0.0]*42
                for i in range(len(voter)):
                    vote[int(voter[i])] += k_top_similarity[i]

                #print(k_top_similarity)
                #print(voter)

                writer.writerow([Wireless_Test_row_label,voter])

                predict_label.append(max(voter, key = voter.count)) #票票等值
                #predict_label.append(vote.index(max(vote))) # 票票不等值 similarity 為權重
                print("Ground Truth :" , Wireless_Test_row_label)
                print("predict_label :",max(voter, key = voter.count))
            
                reference_point = []
                reference_point.append(Wireless_Test_row_label)
                reference_point.append(max(voter, key = voter.count))
                reference_point.append(voter)
                
                wireless_reference_point.writerow(reference_point)
    
    
#計算 MDE
    MDE = 0
    x = [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,11,11,11,11,11,11,11,11,11,11, 1, 2, 3, 4, 5, 6, 7, 8, 9,10,11]
    y = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9,10, 1, 2, 3, 4, 5, 6, 7, 8, 9,10, 1, 2, 3, 4, 5, 6, 7, 8, 9,10,11,11,11,11,11,11,11,11,11,11,11]

    for i in range(len(test_label)):
        x_test_label = x[int(test_label[i])]
        y_test_label = y[int(test_label[i])]
        x_predict_label = x[int(predict_label[i])]
        y_predict_label = y[int(predict_label[i])]
        error = ((x_test_label - x_predict_label)**2 + (y_test_label - y_predict_label)**2)**0.5
        error_distri[math.ceil(error)] += 1
        MDE += error

    total_DE += MDE*0.6
    len_of_all_label += len(test_label)
    MDE = MDE*0.6/len(test_label)
    MDE = round(MDE, 4)
    list_MDE.append(MDE)

    print(methods,"in" ,walk," Mean Distance Error is : ", MDE)    
#計算 MDE      
    
print(list_MDE)
print(walk_list)
MDE = total_DE/len_of_all_label
print("Mean Distance Error of ",methods,"in " ,walk,"is : ",MDE)
print(error_distri)

Walk Method :  one_way_walk_1
Ground Truth : 1.0
predict_label : 1.0
Ground Truth : 2.0
predict_label : 2.0
Ground Truth : 3.0
predict_label : 3.0
Ground Truth : 4.0
predict_label : 3.0
Ground Truth : 5.0
predict_label : 3.0
Ground Truth : 6.0
predict_label : 4.0
Ground Truth : 7.0
predict_label : 5.0
Ground Truth : 8.0
predict_label : 9.0
Ground Truth : 9.0
predict_label : 7.0
Ground Truth : 10.0
predict_label : 8.0
Ground Truth : 31.0
predict_label : 9.0
Ground Truth : 31.0
predict_label : 31.0
Ground Truth : 32.0
predict_label : 7.0
Ground Truth : 33.0
predict_label : 32.0
Ground Truth : 34.0
predict_label : 34.0
Ground Truth : 35.0
predict_label : 17.0
Ground Truth : 36.0
predict_label : 36.0
Ground Truth : 36.0
predict_label : 36.0
Ground Truth : 20.0
predict_label : 17.0
Ground Truth : 19.0
predict_label : 17.0
Ground Truth : 18.0
predict_label : 18.0
Ground Truth : 17.0
predict_label : 16.0
Ground Truth : 16.0
predict_label : 17.0
Ground Truth : 15.0
predict_label : 18.0
Ground 

Ground Truth : 36.0
predict_label : 36.0
Ground Truth : 20.0
predict_label : 17.0
Ground Truth : 19.0
predict_label : 17.0
Ground Truth : 18.0
predict_label : 16.0
Ground Truth : 17.0
predict_label : 17.0
Ground Truth : 16.0
predict_label : 17.0
Ground Truth : 15.0
predict_label : 16.0
Ground Truth : 14.0
predict_label : 15.0
Ground Truth : 13.0
predict_label : 12.0
Ground Truth : 12.0
predict_label : 12.0
Ground Truth : 11.0
predict_label : 11.0
4_Hashing_ALL in one_way_walk_5  Mean Distance Error is :  0.6035
Walk Method :  one_way_walk_6
Ground Truth : 11.0
predict_label : 13.0
Ground Truth : 12.0
predict_label : 12.0
Ground Truth : 13.0
predict_label : 13.0
Ground Truth : 14.0
predict_label : 13.0
Ground Truth : 15.0
predict_label : 16.0
Ground Truth : 16.0
predict_label : 18.0
Ground Truth : 17.0
predict_label : 15.0
Ground Truth : 18.0
predict_label : 18.0
Ground Truth : 19.0
predict_label : 17.0
Ground Truth : 20.0
predict_label : 20.0
Ground Truth : 36.0
predict_label : 19.0
Gr

Ground Truth : 14.0
predict_label : 18.0
Ground Truth : 15.0
predict_label : 15.0
Ground Truth : 16.0
predict_label : 15.0
Ground Truth : 17.0
predict_label : 15.0
Ground Truth : 18.0
predict_label : 18.0
Ground Truth : 19.0
predict_label : 17.0
Ground Truth : 20.0
predict_label : 19.0
Ground Truth : 36.0
predict_label : 36.0
Ground Truth : 36.0
predict_label : 36.0
Ground Truth : 35.0
predict_label : 36.0
Ground Truth : 34.0
predict_label : 34.0
Ground Truth : 33.0
predict_label : 34.0
Ground Truth : 32.0
predict_label : 33.0
Ground Truth : 31.0
predict_label : 31.0
Ground Truth : 31.0
predict_label : 31.0
Ground Truth : 10.0
predict_label : 7.0
Ground Truth : 9.0
predict_label : 9.0
Ground Truth : 8.0
predict_label : 31.0
Ground Truth : 7.0
predict_label : 5.0
Ground Truth : 6.0
predict_label : 6.0
Ground Truth : 5.0
predict_label : 6.0
Ground Truth : 4.0
predict_label : 4.0
Ground Truth : 3.0
predict_label : 2.0
Ground Truth : 2.0
predict_label : 1.0
Ground Truth : 1.0
predict_label

In [8]:
hashing
free
4W_ans,0.5364 0.5224
3W_ans,0.7045
2W_ans,0.6576
1W_ans,1.3560 

scripted_walk
4W_ans,0.5232
3W_ans,0.5751
2W_ans,0.5769
1W_ans,0.9661

stationary
4W_ans,0.4286
3W_ans,0.4285
2W_ans,0.1714
1W_ans,1.5428

knn
free
4W_ans,0.5746484375000034
3W_ans,0.8550772569444456
2W_ans,1.0178585069444415
1W_ans,1.246707465277773

scripted_walk
4W_ans,0.7434443558391465
3W_ans,0.8427749424065699
2W_ans,1.036628477760117
1W_ans,1.2246578947369309

stationary
4W_ans,0.6214285714285713
3W_ans,0.942857142857143
2W_ans,1.4655714285714285
1W_ans,1.4655714285714285

SyntaxError: invalid syntax (<ipython-input-8-c8dae3f6c041>, line 3)

In [ ]:
freewalk
*1
[1.1741, 1.1294, 2.3156, 1.6286, 1.3712, 1.0949, 1.95, 1.2053, 6.3298]
Mean Distance Error of  4_Hashing_ALL in  freewalk_9 is :  1.7702225849111
[31, 28, 31, 7, 8, 1, 5, 2, 1, 0, 0, 11, 7, 0, 0, 0] 
*2
[0.2526, 1.1083, 0.4941, 0.767, 0.3, 0.5625, 0.975, 0.375, 0.975]
Mean Distance Error of  4_Hashing_ALL in  freewalk_9 is :  0.6218465839766621
[61, 37, 15, 11, 6, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0]
*3
[0.3474, 0.6, 0.4941, 0.4435, 0.4875, 0.6375, 0.6, 0.9, 0.675]
Mean Distance Error of  4_Hashing_ALL in  freewalk_9 is :  0.5363636363636364
[73, 24, 21, 8, 3, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.7922
*4
[0.9474, 0.6706, 0.3176, 0.6, 0.375, 0.75, 0.75, 0.525, 0.825]
Mean Distance Error of  4_Hashing_ALL in  freewalk_9 is :  0.6318181818181817
[57, 34, 26, 9, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

scripted_walk
*1
[2.1906, 1.8886, 2.7673, 2.1499, 1.18, 1.1093, 2.8917, 2.4764, 2.5546, 1.4965, 2.5971]
Mean Distance Error of  4_Hashing_ALL in  round_trip_walk_4 is :  2.1475055226154542
[80, 106, 73, 55, 19, 15, 15, 7, 6, 1, 10, 68, 11, 0, 0, 0]
*2
[0.6977, 0.8472, 0.7497, 0.6909, 0.716, 0.7846, 0.6732, 0.6303, 0.7686, 0.6561, 0.583]
Mean Distance Error of  4_Hashing_ALL in  round_trip_walk_4 is :  0.7105575546036568
[167, 140, 104, 25, 18, 6, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0]
*3
[0.7766, 0.6228, 0.8108, 0.6909, 0.6429, 0.6516, 0.5143, 0.5232, 0.6311, 0.7971, 0.6132]
Mean Distance Error of  4_Hashing_ALL in  round_trip_walk_4 is :  0.6707045009616716
[184, 137, 76, 42, 19, 5, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0.5232

stationary
*1
Ground Truth : 21.0
predict_label : 22.0
Ground Truth : 41.0
predict_label : 6.0
Ground Truth : 36.0
predict_label : 6.0
Ground Truth : 16.0
predict_label : 14.0
Ground Truth : 11.0
predict_label : 15.0
Ground Truth : 1.0
predict_label : 1.0
Ground Truth : 31.0
predict_label : 31.0
[2.1644]
Mean Distance Error of  4_Hashing_ALL in  stationary_1 is :  2.1644063742312363
[2, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]
*2
Ground Truth : 21.0
predict_label : 21.0
Ground Truth : 41.0
predict_label : 30.0
Ground Truth : 36.0
predict_label : 20.0
Ground Truth : 16.0
predict_label : 16.0
Ground Truth : 11.0
predict_label : 12.0
Ground Truth : 1.0
predict_label : 1.0
Ground Truth : 31.0
predict_label : 8.0
[0.5143]
Mean Distance Error of  4_Hashing_ALL in  stationary_1 is :  0.5142857142857142
[3, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
*3
Ground Truth : 21.0
predict_label : 21.0
Ground Truth : 41.0
predict_label : 41.0
Ground Truth : 36.0
predict_label : 17.0
Ground Truth : 16.0
predict_label : 15.0
Ground Truth : 11.0
predict_label : 11.0
Ground Truth : 1.0
predict_label : 1.0
Ground Truth : 31.0
predict_label : 31.0
[0.4286]
Mean Distance Error of  4_Hashing_ALL in  stationary_1 is :  0.42857142857142855
[5, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]